In [80]:
import pandas as pd

In [81]:
traffic_Seoul =pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()


,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [82]:
traffic_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      4500 non-null   int64 
 1   월       4500 non-null   int64 
 2   자치구명    4500 non-null   object
 3   발생건수    4500 non-null   int64 
 4   사망자수    4500 non-null   int64 
 5   부상자수    4500 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 211.1+ KB


In [83]:
# 2019년 데이터만 추추라기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul.년도 == 2019].reset_index(drop=True)
traffic_Seoul_2019.head()


,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158


In [84]:
import numpy as np

In [85]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기 
# traffic_analysis = traffic_Seoul_2019.groupby('자치구명').sum().loc[:,['발생건수','부상자수','사망자수']]
traffic_analysis = pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수','부상자수','사망자수']], index='자치구명',aggfunc=np.sum)
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [86]:
seoul_limit = pd.read_csv("../Data/seoul.csv",encoding='euc-kr')
seoul_limit.columns = ['자치구명','lon','lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [87]:
data_result = pd.merge(traffic_analysis,seoul_limit,on='자치구명').set_index('자치구명')
data_result.head()

,발생건수,부상자수,사망자수,lon,lat
자치구명,,,,,
강남구,3722,5182,14,127.0475,37.51731
강동구,1414,1910,11,127.1238,37.53013
강북구,1277,1706,7,127.0255,37.63975
강서구,1829,2491,20,126.8496,37.55094
관악구,1363,1755,10,126.9515,37.47834


In [88]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 강남구 to 중랑구
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   발생건수    25 non-null     int64  
 1   부상자수    25 non-null     int64  
 2   사망자수    25 non-null     int64  
 3   lon     25 non-null     float64
 4   lat     25 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 1.2+ KB


In [89]:
# folium은 int를 모름! 그래서 float로 변환 해야 함
data_result.iloc[:,0:3] = data_result.iloc[:,0:3].astype('float')
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 강남구 to 중랑구
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   발생건수    25 non-null     float64
 1   부상자수    25 non-null     float64
 2   사망자수    25 non-null     float64
 3   lon     25 non-null     float64
 4   lat     25 non-null     float64
dtypes: float64(5)
memory usage: 1.2+ KB


### 자치구의 구청을 표시하기

In [90]:
import folium


In [91]:
data_result.head()

,발생건수,부상자수,사망자수,lon,lat
자치구명,,,,,
강남구,3722.0,5182.0,14.0,127.0475,37.51731
강동구,1414.0,1910.0,11.0,127.1238,37.53013
강북구,1277.0,1706.0,7.0,127.0255,37.63975
강서구,1829.0,2491.0,20.0,126.8496,37.55094
관악구,1363.0,1755.0,10.0,126.9515,37.47834


In [149]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 강남구 to 중랑구
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   발생건수    25 non-null     float64
 1   부상자수    25 non-null     float64
 2   사망자수    25 non-null     float64
 3   lon     25 non-null     float64
 4   lat     25 non-null     float64
dtypes: float64(5)
memory usage: 1.7+ KB


In [176]:
# tiles = 'Stamen Terrain', # Stamen Toner

map = folium.Map(
    location=[37.5502,126.982], 
    tiles = 'Stamen Toner',
    
    zoom_start=11
    )
for name,lon,lat in zip(data_result.index, data_result.lon, data_result.lat):
    folium.Marker([lat,lon],popup=name).add_to(map)
    
map


In [177]:
data_result.head(1)

,발생건수,부상자수,사망자수,lon,lat
자치구명,,,,,
강남구,3722.0,5182.0,14.0,127.0475,37.51731


In [178]:
# 자치구별 교통사고 발생건수를 표시하기
for name, lat, lng,count,deathcount in zip(data_result.index, data_result.lat, data_result.lon,data_result.발생건수,data_result.사망자수):
    # folium.CircleMarker(
    #     [lat,lng],
    #     radius = (count / data_result.발생건수.max())*30, # 원의 반지름
    #     color = 'blue',
    #     fill = True,
    #     fill_color = 'skyblue',
    #     fill_opacity = 0.1,
    #     popup = name
    #     ).add_to(map)
    if((deathcount ==  data_result.사망자수.max()) |   (deathcount == data_result.사망자수.min())):
        folium.CircleMarker(
        [lat,lng],
        radius = (deathcount / data_result.사망자수.max())*30, # 원의 반지름
        color = 'red',
        fill = True,
        fill_color = '레드',
        fill_opacity = 0.1,
        popup = name
        ).add_to(map)
        


map

,발생건수,부상자수,사망자수,lon,lat
자치구명,,,,,
강남구,3722.0,5182.0,14.0,127.0475,37.51731
강동구,1414.0,1910.0,11.0,127.1238,37.53013
강북구,1277.0,1706.0,7.0,127.0255,37.63975
강서구,1829.0,2491.0,20.0,126.8496,37.55094
관악구,1363.0,1755.0,10.0,126.9515,37.47834
